In [9]:
import time
import numpy as np
import matplotlib.pyplot as plt
from ripser import ripser
from persim import plot_diagrams
from sklearn.cluster import KMeans
import tadasets
from itertools import combinations
%matplotlib inline

In [13]:
#Functions used

def kmeans_downsampling(data, num_points): #Sampling
    kmeans = KMeans(n_clusters=num_points, random_state=0).fit(data)
    centroids = kmeans.cluster_centers_
    return centroids
num_points = 100

def cupProduct(phi, psi, triangleList): #Compute Cup Product
    cupProduct = []
    for i in range(len(triangleList)):
        phiVal = 0
        psiVal = 0
        for j in range(len(phi)):
            if phi[j][0] == triangleList[i][0] and phi[j][1] == triangleList[i][1]:
                phiVal = phi[j][2]
                break;
        for k in range(len(psi)):
            if psi[k][0] == triangleList[i][1] and psi[k][1] == triangleList[i][2]:
                psiVal = psi[k][2]
                break;
        cupProduct.append(phiVal * psiVal)
    cupProduct = np.array(cupProduct).T
    return cupProduct

def reductionAlgorithm(R):
    numRows = len(R[0])
    V = np.eye(numRows)
    pivotRows, pivotEntries = [[] for i in range (numRows)], [[] for i in range (numRows)]

    for i in range(numRows):
        h = 0
        while (h == 0):
            pivotRow = np.inf
            for j in range(numRows - 1, -1, -1):
                if R[j][i] != 0:
                    pivotRow = j
                    break;
            if pivotRow == np.inf:
                pivotEntries[i] = 0
            else:
                pivotEntries[i] = R[pivotRow][i]
            pivotRows[i] = pivotRow
            if pivotRow == np.inf or all(pivotRows[k] != pivotRows[i] for k in range(i)):
                h = 1
                break;
            for k in range(i):
                if pivotRows[k] == pivotRows[i]:
                    c = pivotEntries[i] // pivotEntries[k]
                    for l in range(numRows):
                        R[l][i] -= c * R[l][k]
                        V[l][i] -= c * V[l][k]            
    return R

def checkSolution(A, b):
    # Calculate the rank of the augmented matrix [A | b]
    augmented_matrix = np.column_stack((A, b))
    rank_A = np.linalg.matrix_rank(A)
    rank_augmented = np.linalg.matrix_rank(augmented_matrix)
    num_columns = A.shape[1]
    num_rows = A.shape[0]
    if rank_A == rank_augmented:
        if rank_A == num_columns:
            return 1
        elif rank_A < num_columns:
            return 1
    else:
        if rank_A < rank_augmented:
            return 0
    return "Unable to determine the solution status."

def rowDetect(A, b): #restricts the matrix and checks for when the row has no soultions
    iMax = len(A) + 1
    iMin = 1
    i = len(A) // 2
    running = True
    while(running):
        print(i)
        sol = checkSolution(A[-i:], b[-i:])
        if sol == 0:
            if checkSolution(A[-(i - 1):], b[-(i - 1):]) == 1:
                row = len(A) - i
                running = False
                break;
            else:
                if i < iMax:
                    iMax = i
                    i = (iMax + iMin) // 2
        if sol == 1:
            if i > iMin:
                iMin = i
                i = (iMax + iMin) // 2  
            if i == len(A): 
                row = -1 
                running = False
                break;
    return row+1

def rowTriangle(r):
    value=len(triangles)-1-r
    return triangles[value]

In [3]:
#Generate a shape

np.random.seed(2) #This for a torus
n_data = 25000
R = 5
r = 2
data = np.zeros((3, n_data))
s = np.random.rand(n_data)*2*np.pi
t = np.random.rand(n_data)*2*np.pi
data[0] = (R + r*np.cos(s))*np.cos(t)
data[1] = (R + r*np.cos(s))*np.sin(t)
data[2] = r*np.sin(s)
data += 0.1*np.random.randn(*data.shape)

In [4]:
start_time=time.time()

#Part 1: Get the Cup Product
x = kmeans_downsampling(data.T, 100)
result = ripser(x, coeff=2, do_cocycles=True)
diagrams = result['dgms']
cocycles = result['cocycles'] #all of the cocycles
D = result['dperm2all'] #distance matrix between the ith and jth points in the data
dgm1 = diagrams[1]
#Representative cocycle phi
idx1 = np.argmax(dgm1[:, 1] - dgm1[:, 0])
cocycle1 = cocycles[1][idx1]
#Representative cocycle psi
sorted_indices = np.argsort(dgm1[:, 1] - dgm1[:, 0])
idx2 = sorted_indices[-2]
cocycle2 = cocycles[1][idx2]

#Restrict cocycle1
New_cocycle1=[]
edges=[]
for i in range(len(cocycle1)):
        for j in range(len(cocycle2)):
            if np.array_equal(cocycle1[i:i-1, :2], cocycle1[i:i+1, :2], cocycle2[j:j+1, :2]):  
                New_cocycle1.append(cocycle2[j:j+1])
        New_cocycle1.append(cocycle2[i:i+1])        
rcocycle1=np.vstack(New_cocycle1)
for i in range(len(cocycle1)):
        for j in range(len(cocycle2)):
            if np.array_equal(cocycle1[i-1:i, :2], cocycle2[j:j+1, :2]):
                edges.append(cocycle2[j:j+1])        
edges=np.vstack(edges)
for i in range(len(rcocycle1)):
    found_match = False
    for j in range(len(edges)):
        if np.array_equal(rcocycle1[i, :2], edges[j, :2]):
            found_match = True
            break
    if not found_match:
        rcocycle1[i, -1] = 0
        
# Finding the representative cocycles for triangles
representative_cocycles = []
for i in range(len(dgm1)):
    birth, death = dgm1[i]
    if birth != death:  # ignore points on the diagonal
        cocycle = cocycles[1][i]
        representative_cocycles.append(cocycle)
triangles = []
for cocycle in representative_cocycles:
    edge_indices = cocycle[:, :2].astype(int)
    triangle_vertices = set()
    for i, j in edge_indices:
        triangle_vertices.add(i)
        triangle_vertices.add(j)
    triangle_combinations = combinations(triangle_vertices, 3)
    for combination in triangle_combinations:
        triangles.append(list(combination))
t=np.vstack(triangles)

cup=cupProduct(rcocycle1,cocycle2, t)#Compute the cup Product
cup=np.vstack(cup)



/Users/sofiamartinez/opt/anaconda3/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [5]:
#Part 2: Get the Coboundary Matrix
edges = []
vertices=[]
representative_cocycles = []
for i in range(len(dgm1)):
    birth, death = dgm1[i]
    if birth != death:  
        cocycle = cocycles[1][i]
        representative_cocycles.append(cocycle)
for cocycle in representative_cocycles: # Extracting every edge as an array of vertices
    edge_indices = cocycle[:, :2].astype(int)
    for i, j in edge_indices:
        edge = [i, j]
        edges.append(edge)
for cocycle in representative_cocycles: #Extracting every vertex
    vertex_indices = cocycle[:, :1].astype(int)
    for i in vertex_indices:
        vertices.append(i[0])
vertices = list(set(vertices)) # Remove duplicate vertices by converting the list to a set and then back to a list

ne = len(edges)
nt = len(triangles)
nv = len(vertices)
num_rows= ne+nt+nv
num_cols=ne+nt+nv
boundary_matrix=np.zeros((num_rows, num_cols), dtype=int)
for i,edge in enumerate(edges):
    a,b=edge #a and b are the two verticies that make up the edge
    for j,vertex in enumerate(vertices):
        c=vertex # c is a vertex
        if b == c:
            boundary_matrix[nv+i,j]=1
        if a == c:
            boundary_matrix[nv+i,j]=-1
for i, triangle in enumerate(triangles):
    e,f,g=triangle #e,f,g are verticies in the triangle
    for j,edge in enumerate(edges):
        h,k=edge 
        if (h,k) == (e,f):
            boundary_matrix[(ne+nv)+i,nv+j]=1
        if (h,k) == (f,g):
            boundary_matrix[(ne+nv)+i,nv+j]=1
        if (h,k) == (e,g):
            boundary_matrix[(ne+nv)+i,nv+j]=-1
boundary_matrix=boundary_matrix.T
#Convert Boundary Matrix into Coboundary
restricted_matrix = boundary_matrix[nv:nv + ne, nv + ne:len(boundary_matrix)]
coboundary_matrix = np.flip(restricted_matrix).T

In [14]:
#Part 3: Solve for x
reduce=reductionAlgorithm(coboundary_matrix) #Reduce Coboundary Matrix
detect=rowDetect(reduce, cup) #detect row when matrix no longer has a solution
tri=rowTriangle(detect) #Obtain associated triangle with the detected row
sorted_indices = np.argsort(dgm1[:, 1] - dgm1[:, 0])
idx = sorted_indices[-2]
h = max(dgm1[idx, 0], dgm1[idx, 0])
while h < min(dgm1[idx, 1], dgm1[idx, 1]):
    newRun = ripser(x, coeff=2, thresh = h, do_cocycles = True)
    newDiagrams = newRun['dgms']
    newCocycles = newRun['cocycles']
    D = newRun['dperm2all']
    dgmNew = newDiagrams[1]
    new_representative_cocycles = []
    for i in range(len(dgmNew)):
        birth, death = dgmNew[i]
        if birth != death:  # ignore points on the diagonal
            newCocycle = newCocycles[1][i]
            new_representative_cocycles.append(newCocycle)
    newTriangles = []
    for newCocycle in new_representative_cocycles:
        edge_indices = newCocycle[:, :2].astype(int)
        new_triangle_vertices = set()
        for i, j in edge_indices:
            new_triangle_vertices.add(i)
            new_triangle_vertices.add(j)
        new_triangle_combinations = combinations(new_triangle_vertices, 3)
        for combination in new_triangle_combinations:
            newTriangles.append(list(combination))
    t=np.vstack(newTriangles)
    for k in range(len(t)):
        if t[k][0] == tri[0] and t[k][1] == tri[1] and t[k][2] == tri[2]:
            death=h #Gives the Cohomological death 
            h = 1000000
            break;
    h += 0.1

38826
58239
67946
72799
75226
76439
77046
77349
77501
77577
77615
77634
77643
77648
77650
77651


ValueError: need at least one array to concatenate

In [ ]:
#Part 4: Plot the persistence diagram with the new point
birth=dgm1[idx2,1] #death of psi but birth of H2 point 
plot_diagrams(diagrams, show = False)
plt.scatter(dgm1[0], dgm1[1], 20)
plt.scatter(death,birth, color='g')
plt.title("Peristence Diagram")
plt.show()
end_time=(time.time()-start_time)
print("Time Elapsed =",end_time)

In [12]:
len(reduce)

77652